# Работа с текстами на русском языке

В языках мира существуют две основных группы способов выражения грамматических значений: 

- синтетические способы
- аналитические способы

Для **синтетических способов** характерно соединение грамматического показателя с самим словом. Таким показателем, вносящим грамматическое значение ≪внутрь слова≫, могут быть окончание, суффикс, приставка, внутренняя флексия и т.д.

Общей чертой **аналитических способов** является выражение грамматического значения за пределами слова, отдельно от него — например, с помощью предлогов, союзов, артиклей, вспомогательных глаголов и других служебных слов, а также с помощью порядка слов.

Существеное отличие в работе с русским и английским текстом заключается в том, что для русского языка более хврактерен синтетический способ, а для английского -- аналитический.

### Постановка задачи

При работе с большим объёмом данных важно поддерживать их чистоту. А при заполнении заявки на банковский продукт необходимо указывать полные паспортные данные, в том числе и поле «кем выдан паспорт», число различных вариантов написаний одного и того же отделения потенциальными клиентами может достигать нескольких сотен. Важно понимать, не ошибся ли клиент, заполняя другие поля: «код подразделения», «серию/номер паспорта». Для этого необходимо сверять «код подразделения» и «кем выдан паспорт».
Задача заключается в том, чтобы проставить коды подразделений для записей из тестовой выборки, основываясь на обучающей выборке.

https://habr.com/ru/post/205360/

#### Предварительная обработка данных

Загрузим данные и посмотрим, что мы имеем:

In [1]:
from pandas import read_csv
import pymorphy2
from sklearn.feature_extraction.text import HashingVectorizer
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, roc_auc_score
from sklearn.decomposition import PCA

train = read_csv('https://static.tcsbank.ru/documents/olymp/passport_training_set.csv',';', index_col='id' ,encoding='cp1251')
train.head(5)

,passport_div_code,passport_issuer_name,passport_issue_month/year
id,,,
1,422008,БЕЛОВСКИМ УВД КЕМЕРОВСКОЙ ОБЛАСТИ,11M2001
2,500112,ТП №2 В ГОР. ОРЕХОВО-ЗУЕВО ОУФМС РОССИИ ПО МО ...,03M2009
3,642001,ВОЛЖСКИМ РОВД ГОР.САРАТОВА,04M2002
4,162004,УВД МОСКОВСКОГО РАЙОНА Г.КАЗАНЬ,12M2002
5,80001,ОТДЕЛОМ ОФМС РОССИИ ПО РЕСП КАЛМЫКИЯ В Г ЭЛИСТА,08M2009


Теперь можно посмотреть как пользователи записывают поле «кем выдан паспорт» на примере какого-либо подразделения:


In [2]:
example_code = train.passport_div_code[train.passport_div_code.duplicated()].values[0]
for i in train.passport_issuer_name[train.passport_div_code == example_code].drop_duplicates():
    print(i)

ОТДЕЛЕНИЕМ УФМС РОССИИ ПО РЕСПУБЛИКЕ КАРЕЛИЯ В МЕДВЕЖ. Р-Е
ОТДЕЛЕНИЕМ УФМС РОССИИ ПО Р. КАРЕЛИЯ В МЕДВЕЖЬЕГОРСКОМ РАЙОНЕ
ОТДЕЛЕНИЕМ УФМС РОССИИ ПО РЕСП КАРЕЛИЯ В МЕДВЕЖЬЕГОРСКОМ Р-НЕ
ОТДЕЛЕНИЕМ УФМС РОССИИ ПО РЕСПУБЛИКЕ КАРЕЛИЯ В МЕДВЕЖЬЕГОРСКОМ РАЙОНЕ
ОУФМС РОССИИ ПО РЕСПУБЛИКЕ КАРЕЛИЯ В МЕДВЕЖЬЕГОРСКОМ РАЙОНЕ
УФМС РОССИИ ПО РК В МЕДВЕЖЬЕГОРСКОМ РАЙОНЕ
ОТДЕЛЕНИЕМ УФМС РОССИИ ПО РЕСПУБЛИКЕ КАРЕЛИЯ МЕДВЕЖЬЕГОРСКОМ Р-ОНЕ
ОТДЕЛЕНИЕМ УФМС РОССИИ ПО РК В МЕДВЕЖЬЕГОРСКОМ РАЙОНЕ
ОТДЕЛЕНИЕМ УФМС РОССИИ ПО РЕСПУБЛИКЕ КОРЕЛИЯ В МЕДВЕЖИГОРСКОМ РАЙОНЕ
УФМС РОССИИ ПО Р. КАРЕЛИЯ МЕДВЕЖЬЕГОРСКОГО Р-НА
ОТДЕЛОМ УФМС РОССИИ ПО РЕСПУБЛИКЕ КАРЕЛИЯ В МЕДВЕЖЬЕГОРСКОМ
УФМС РЕСПУБЛИКИ КАРЕЛИИ МЕДВЕЖЬЕГОРСКОГО Р-ОН
МЕДВЕЖЬЕГОРСКИМ ОВД


Как можно заметить нужно на поле действительно заполняется криво. Но для нормально кодирования мы должны привести это поле к более-менее нормальному (однозначному) виду.
Для начала я бы предложил привести все записи к одному регистру, например, чтобы все буквы стали строчными. Это легко сделать с помощью атрибута str, столбца DataFrame'a. Этот атрибут позволяет работать со столбцом как с строкой, а также выполнять различного рода поиск и замену по регулярным выражениям:

In [3]:
train.passport_issuer_name = train.passport_issuer_name.str.lower()
train[train.passport_div_code == example_code].head(5)

,passport_div_code,passport_issuer_name,passport_issue_month/year
id,,,
19,100010,отделением уфмс россии по республике карелия в...,04M2008
22,100010,отделением уфмс россии по р. карелия в медвежь...,10M2009
5642,100010,отделением уфмс россии по респ карелия в медве...,08M2008
6668,100010,отделением уфмс россии по республике карелия в...,08M2011
8732,100010,отделением уфмс россии по республике карелия в...,08M2012


C регистром определились. Далее надо по возможности избавиться от популярных сокращений, например район, город и т.д. Сделаем это с помощью регулярных выражений. Pandas предоставляет удобное использование регулярных выражений применительно к каждому столбцу. Это выглядит так:

In [4]:
train.passport_issuer_name = train.passport_issuer_name.str.replace(u'р-(а|й|о|н|е)*',u'район')
train.passport_issuer_name = train.passport_issuer_name.str.replace(u' г( |\.|(ор(\.| )))', u' город ')
train.passport_issuer_name = train.passport_issuer_name.str.replace(u' р(\.|есп )', u' республика ')
train.passport_issuer_name = train.passport_issuer_name.str.replace(u' адм([а-я]*)(\.)?', u' административный ')
train.passport_issuer_name = train.passport_issuer_name.str.replace(u' окр(\.| |уга( )?)', u' округ ')
train.passport_issuer_name = train.passport_issuer_name.str.replace(u' ао ', u' административный округ ')

Теперь избавимся от всех лишних символов, кроме русских букв, дефисов и пробелов. Это связано с тем, что паспорт о одинаковым подразделением может выдаваться отделами с разными номерами, и это ухудшит дальнейшую кодировку:

In [5]:
train.passport_issuer_name = train.passport_issuer_name.str.replace(u' - ?', u'-')
train.passport_issuer_name = train.passport_issuer_name.str.replace(u'[^а-я -]','')
train.passport_issuer_name = train.passport_issuer_name.str.replace(u'- ',' ')
train.passport_issuer_name = train.passport_issuer_name.str.replace(u'  *',' ')

На следующем шаге, надо расшифровать аббревиатуры, типа УВД, УФНС, ЦАО, ВАО и т.д., т.к. этих их в принципе не много, но на качестве дальнейшего кодирования это скажется положительно. Например если у нас будет две записи «УВД» и «управление внутренних дел», то закодированы они будут по разному, т. к. для компьютера это разные значения.
Итак перейдем к расшифровке. И, для начала, заведем словарь сокращений, с помощью которого мы и сделаем расшифровку:

In [6]:
sokr = {u'нао': u'ненецкий автономный округ',
u'хмао': u'ханты-мансийский автономный округ',
u'чао': u'чукотский автономный округ',
u'янао': u'ямало-ненецкий автономный округ',
u'вао': u'восточный административный округ',
u'цао': u'центральный административный округ',
u'зао': u'западный административный округ',
u'cао': u'северный административный округ',
u'юао': u'южный административный округ',
u'юзао': u'юго-западный округ',
u'ювао': u'юго-восточный округ',
u'свао': u'северо-восточный округ',
u'сзао': u'северо-западный округ',
u'оуфмс': u'отдел управление федеральной миграционной службы',
u'офмс': u'отдел федеральной миграционной службы',
u'уфмс': u'управление федеральной миграционной службы',
u'увд': u'управление внутренних дел',
u'ровд': u'районный отдел внутренних дел',
u'говд': u'городской отдел внутренних дел',
u'рувд': u'районное управление внутренних дел',
u'овд': u'отдел внутренних дел',
u'оувд': u'отдел управления внутренних дел',
u'мро': u'межрайонный отдел',
u'пс': u'паспортный стол',
u'тп': u'территориальный пункт'}

Теперь, собственно произведем расшифровку абривеатур и отформатируем полученные записи:

In [7]:
for key in sokr:
    train.passport_issuer_name = train.passport_issuer_name.str.replace(u'( %s )|(^%s)|(%s$)' % (i,i,i), u' %s ' % (sokr[key]))
    
#удалим лишние пробелы в конце и начале строки
train.passport_issuer_name = train.passport_issuer_name.str.lstrip()
train.passport_issuer_name = train.passport_issuer_name.str.rstrip()

Предварительный этап обработки поля «кем выдан паспорт» на этом закончим. И перейдем к полю, в котором находится дата выдачи.
Как можно заметить данные в нем хранятся в виде: месяцMгод.
Соответственно можно просто убрать букву «M» и привести поле к числовому типу. Но если хорошо подумать, то это поле можно удалить, т.к. на один месяц в году может приходиться несколько подразделений выдававших паспорт, и соответственно это может испортить нашу модель. Исходя из этого удалим его из выборки:


In [8]:
train = train.drop(['passport_issue_month/year'], axis=1)

#### Анализ данных

Итак, данные для построения модели у нас есть, но они находятся в текстовом виде. Для построения модели хорошо бы было их закодировать в числовом виде.

Авторы пакета scikit-learn заботливо о нас позаботились и добавили несколько способов для извлечения и кодирования текстовых данных:

- FeatureHasher
- CountVectorizer
- HashingVectorizer

**FeatureHasher** преобразовывает строку в числовой массив заданной длинной с помощью хэш-функции (32-разрядная версия Murmurhash3)

**CountVectorizer** преобразовывает входной текст в матрицу, значениями которой, являются количества вхождения данного ключа(слова) в текст. В отличие от FeatureHasher имеет больше настраиваемых параметров(например можно задать токенизатор), но работает медленнее.

**HashingVectorizer** является смесью двух выше описанных методов. В нем можно и регулировать размер закодированной строки (как в FeatureHasher) и настраивать токенизатор (как в CountVectorizer). К тому же его производительность ближе к FeatureHasher.

Для начала CountVectorizer собирает уникальные ключи из всех записей.

Длина списка из уникальных ключей и будет длиной нашего закодированного текста. А номера элементов будут соответствовать, количеству раз встречи данного ключа с данным номером в строке.

**Пример:**

Дано:
- раз два три
- три четыре два два
- раз раз раз четыре

Список уникальных ключей: [раз, два, три, четыре]

- раз два три --> [1,1,1,0]
- три четыре два два --> [0,2,1,1]


Итак, вернемся к анализу. Если мы посмотрим по внимательнее на наш набор данных то можно заметить, что есть похожие строки но записанные по разному например: "… республика карелия..." и "… по республике карелия...".

Соответственно, если мы попробуем применить один из методов кодирования сейчас мы получим очень похожие значения. Такие случаем можно минимизировать если все слова в записи мы приведем к нормальной форме.

Для этой задачи хорошо подходит pymorphy или nltk. Я буду использовать первый, т.к. он изначально создавался для работы с русским языком. Итак, функция которая будет отвечать за нормализацию и очиску строки выглядит так:

In [9]:
def f_tokenizer(s):
    morph = pymorphy2.MorphAnalyzer()
    if type(s) == "unicode":
        t = s.split(' ')
    else:
        t = s
    f = []
    for j in t:
        m = morph.parse(j.replace('.',''))
        if len(m) != 0:
            wrd = m[0]
            if wrd.tag.POS not in ('NUMR','PREP','CONJ','PRCL','INTJ'):
                f.append(wrd.normal_form)
    return(f)

Функция делает следующее:
- Сначала она преобразовывает строку в список
- Затем для всех слов производит разбор
- Если слово является числительным, предикативном, предлогом, союзом, частицей или междометием не включаем его в конечный набор
- Если слово не попало в предыдущий список, берем его нормальную форму и добавляем в финальный набор

Теперь, когда есть функция для нормализации можно приступить к кодированию с помощью метода CountVectorizer. Он выбран потому, что ему можно передать нашу функцию, как токенизатор и он составит список ключей по значениям полученным в результате работы нашей функции:


In [10]:
coder = HashingVectorizer(tokenizer=f_tokenizer, n_features=256)

Как можно заметить при создании метода кроме токенизатора мы задаем еще один параметр n_features. Через данный параметр задается длина закодированной строки (в нашем случае строка кодируется при помощи 256 столбцов). Кроме того, у HashingVectorizer есть еще одно преимущество перед CountVectorizer, но сразу может выполнять нормализацию значений, что хорошо для таких алгоритмов, как SVM.
Теперь применим наш кодировщик к обучающему набору:

In [11]:
TrainNotDuble = train.drop_duplicates()
trn = coder.fit_transform(TrainNotDuble.passport_issuer_name.tolist()).toarray()

#### Построение модели

Для начала нам надо задать значения для столбца, в котором будут содержаться метки классов:

In [12]:
target = TrainNotDuble.passport_div_code.values

In [13]:
pca = PCA(n_components = 15)
trn = pca.fit_transform(trn)

In [17]:
model = RandomForestClassifier(n_estimators = 100, criterion='entropy')

TRNtrain, TRNtest, TARtrain, TARtest = train_test_split(trn, target, test_size=0.4)
model.fit(TRNtrain, TARtrain)
print('accuracy_score: ', accuracy_score(TARtest, model.predict(TRNtest)))

accuracy_score:  0.7379523720232515


# Анализ тональности текстов с помощью сверточных нейронных сетей

<img width=600 src="net.gif">
https://habr.com/ru/company/mailru/blog/417767/

Представьте, что у вас есть абзац текста. Можно ли понять, какую эмоцию несет этот текст: радость, грусть, гнев? Можно. Упростим себе задачу и будем классифицировать эмоцию как позитивную или как негативную, без уточнений. Есть много способов решать такую задачу, и один из них — свёрточные нейронные сети (Convolutional Neural Networks). CNN изначально были разработаны для обработки изображений, однако они успешно справляются с решением задач в сфере автоматической обработки текстов. 

Для обучения я выбрал корпус коротких текстов Юлии Рубцовой, сформированный на основе русскоязычных сообщений из Twitter. Он содержит 114 991 положительных, 111 923 отрицательных твитов, а также базу неразмеченных твитов объемом 17 639 674 сообщений.

In [1]:
import pandas as pd
import numpy as np

# Считываем данные
n = ['id', 'date', 'name', 'text', 'typr', 'rep', 'rtw', 'faw', 'stcount', 'foll', 'frien', 'listcount']
data_positive = pd.read_csv('data/positive.csv', sep=';', error_bad_lines=False, names=n, usecols=['text'])
data_negative = pd.read_csv('data/negative.csv', sep=';', error_bad_lines=False, names=n, usecols=['text'])

# Формируем сбалансированный датасет
sample_size = min(data_positive.shape[0], data_negative.shape[0])
raw_data = np.concatenate((data_positive['text'].values[:sample_size],
                           data_negative['text'].values[:sample_size]), axis=0)
labels = [1] * sample_size + [0] * sample_size

Перед началом обучения тексты прошли процедуру предварительной обработки:

- приведение к нижнему регистру;
- замена «ё» на «е»;
- замена ссылок на токен «URL»;
- замена упоминания пользователя на токен «USER»;
- удаление знаков пунктуации.

In [13]:
import re

def preprocess_text(text):
    text = text.lower().replace("ё", "е")
    text = re.sub('((www\.[^\s]+)|(https?://[^\s]+))', 'URL', text)
    text = re.sub('@[^\s]+', 'USER', text)
    text = re.sub('[^a-zA-Zа-яА-Я1-9]+', ' ', text)
    text = re.sub(' +', ' ', text)
    return text.strip()


data = [preprocess_text(t) for t in raw_data]

In [14]:
from sklearn.model_selection import train_test_split

x_train, x_test, y_train, y_test = train_test_split(data, labels, test_size=0.2, random_state=1)

#### Векторное отображение слов

Входными данными сверточной нейронной сети является матрица с фиксированной высотой n, где каждая строка представляет собой векторное отображение слова в признаковое пространство размерности k.

Для формирования embedding-слоя нейронной сети используем утилиту дистрибутивной семантики Word2Vec, предназначенную для отображения семантического значения слов в векторное пространство. 

Word2Vec находит взаимосвязи между словами согласно предположению, что в похожих контекстах встречаются семантически близкие слова. Подробнее о Word2Vec можно прочитать в оригинальной статье, а также тут и тут. Поскольку твитам характерна авторская пунктуация и эмотиконы, определение границ предложений становится достаточно трудоемкой задачей. 

In [15]:
import sqlite3

# Открываем SQLite базу данных
conn = sqlite3.connect('mysqlite3.db')
c = conn.cursor()

In [17]:
with open('data/tweets.txt', 'w', encoding='utf-8') as f:
    # Считываем тексты твитов 
    for row in c.execute('SELECT ttext FROM sentiment'):
        if row[0]:
            tweet = preprocess_text(row[0])
            # Записываем предобработанные твиты в файл
            print(tweet, file=f)

Далее с помощью библиотеки Gensim обучаем Word2Vec-модель со следующими параметрами: 

- size = 200 — размерность признакового пространства;
- window = 5 — количество слов из контекста, которое анализирует алгоритм;
- min_count = 3 — слово должно встречаться минимум три раза, чтобы модель его учитывала.

In [18]:
import logging
import multiprocessing
import gensim
from gensim.models import Word2Vec

logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)
# Считываем файл с предобработанными твитами
data = gensim.models.word2vec.LineSentence('data/tweets.txt')
# Обучаем модель 
model = Word2Vec(data, size=200, window=5, min_count=3, workers=multiprocessing.cpu_count())


2019-08-14 07:13:33,867 : INFO : collecting all words and their counts
2019-08-14 07:13:33,870 : INFO : PROGRESS: at sentence #0, processed 0 words, keeping 0 word types
2019-08-14 07:13:33,954 : INFO : PROGRESS: at sentence #10000, processed 98345 words, keeping 24248 word types
2019-08-14 07:13:34,026 : INFO : PROGRESS: at sentence #20000, processed 194782 words, keeping 38847 word types
2019-08-14 07:13:34,098 : INFO : PROGRESS: at sentence #30000, processed 290743 words, keeping 50531 word types
2019-08-14 07:13:34,167 : INFO : PROGRESS: at sentence #40000, processed 386269 words, keeping 60653 word types
2019-08-14 07:13:34,231 : INFO : PROGRESS: at sentence #50000, processed 482755 words, keeping 69689 word types
2019-08-14 07:13:34,305 : INFO : PROGRESS: at sentence #60000, processed 578104 words, keeping 78235 word types
2019-08-14 07:13:34,375 : INFO : PROGRESS: at sentence #70000, processed 672686 words, keeping 85840 word types
2019-08-14 07:13:34,474 : INFO : PROGRESS: at s

2019-08-14 07:13:39,197 : INFO : PROGRESS: at sentence #720000, processed 6856226 words, keeping 329362 word types
2019-08-14 07:13:39,261 : INFO : PROGRESS: at sentence #730000, processed 6953689 words, keeping 331913 word types
2019-08-14 07:13:39,335 : INFO : PROGRESS: at sentence #740000, processed 7054203 words, keeping 334539 word types
2019-08-14 07:13:39,403 : INFO : PROGRESS: at sentence #750000, processed 7153523 words, keeping 336844 word types
2019-08-14 07:13:39,483 : INFO : PROGRESS: at sentence #760000, processed 7253230 words, keeping 339124 word types
2019-08-14 07:13:39,552 : INFO : PROGRESS: at sentence #770000, processed 7352424 words, keeping 341418 word types
2019-08-14 07:13:39,623 : INFO : PROGRESS: at sentence #780000, processed 7450987 words, keeping 343877 word types
2019-08-14 07:13:39,699 : INFO : PROGRESS: at sentence #790000, processed 7545233 words, keeping 346595 word types
2019-08-14 07:13:39,763 : INFO : PROGRESS: at sentence #800000, processed 763865

2019-08-14 07:13:44,536 : INFO : PROGRESS: at sentence #1430000, processed 13698594 words, keeping 494012 word types
2019-08-14 07:13:44,613 : INFO : PROGRESS: at sentence #1440000, processed 13799436 words, keeping 496301 word types
2019-08-14 07:13:44,686 : INFO : PROGRESS: at sentence #1450000, processed 13898908 words, keeping 498556 word types
2019-08-14 07:13:44,764 : INFO : PROGRESS: at sentence #1460000, processed 13996483 words, keeping 500687 word types
2019-08-14 07:13:44,836 : INFO : PROGRESS: at sentence #1470000, processed 14094360 words, keeping 502819 word types
2019-08-14 07:13:44,908 : INFO : PROGRESS: at sentence #1480000, processed 14192679 words, keeping 504855 word types
2019-08-14 07:13:45,001 : INFO : PROGRESS: at sentence #1490000, processed 14291910 words, keeping 507157 word types
2019-08-14 07:13:45,089 : INFO : PROGRESS: at sentence #1500000, processed 14391159 words, keeping 509297 word types
2019-08-14 07:13:45,160 : INFO : PROGRESS: at sentence #1510000,

2019-08-14 07:13:49,945 : INFO : PROGRESS: at sentence #2140000, processed 20623466 words, keeping 625917 word types
2019-08-14 07:13:50,025 : INFO : PROGRESS: at sentence #2150000, processed 20722248 words, keeping 627462 word types
2019-08-14 07:13:50,094 : INFO : PROGRESS: at sentence #2160000, processed 20819863 words, keeping 629122 word types
2019-08-14 07:13:50,180 : INFO : PROGRESS: at sentence #2170000, processed 20917554 words, keeping 630708 word types
2019-08-14 07:13:50,251 : INFO : PROGRESS: at sentence #2180000, processed 21015641 words, keeping 632393 word types
2019-08-14 07:13:50,334 : INFO : PROGRESS: at sentence #2190000, processed 21113827 words, keeping 634124 word types
2019-08-14 07:13:50,403 : INFO : PROGRESS: at sentence #2200000, processed 21212464 words, keeping 635844 word types
2019-08-14 07:13:50,489 : INFO : PROGRESS: at sentence #2210000, processed 21310897 words, keeping 637585 word types
2019-08-14 07:13:50,569 : INFO : PROGRESS: at sentence #2220000,

2019-08-14 07:13:55,355 : INFO : PROGRESS: at sentence #2850000, processed 27587700 words, keeping 742350 word types
2019-08-14 07:13:55,439 : INFO : PROGRESS: at sentence #2860000, processed 27686628 words, keeping 744033 word types
2019-08-14 07:13:55,517 : INFO : PROGRESS: at sentence #2870000, processed 27788789 words, keeping 745708 word types
2019-08-14 07:13:55,595 : INFO : PROGRESS: at sentence #2880000, processed 27885614 words, keeping 747258 word types
2019-08-14 07:13:55,660 : INFO : PROGRESS: at sentence #2890000, processed 27979645 words, keeping 748829 word types
2019-08-14 07:13:55,745 : INFO : PROGRESS: at sentence #2900000, processed 28075614 words, keeping 750449 word types
2019-08-14 07:13:55,815 : INFO : PROGRESS: at sentence #2910000, processed 28172950 words, keeping 752004 word types
2019-08-14 07:13:55,888 : INFO : PROGRESS: at sentence #2920000, processed 28270410 words, keeping 753472 word types
2019-08-14 07:13:55,959 : INFO : PROGRESS: at sentence #2930000,

...

2019-08-14 07:45:34,299 : INFO : EPOCH 5 - PROGRESS: at 30.13% examples, 379838 words/s, in_qsize 2, out_qsize 0
2019-08-14 07:45:35,317 : INFO : EPOCH 5 - PROGRESS: at 30.39% examples, 379792 words/s, in_qsize 6, out_qsize 1
2019-08-14 07:45:36,338 : INFO : EPOCH 5 - PROGRESS: at 30.69% examples, 380152 words/s, in_qsize 0, out_qsize 0
2019-08-14 07:45:37,349 : INFO : EPOCH 5 - PROGRESS: at 30.98% examples, 380399 words/s, in_qsize 0, out_qsize 0
2019-08-14 07:45:38,352 : INFO : EPOCH 5 - PROGRESS: at 31.21% examples, 380026 words/s, in_qsize 0, out_qsize 0
2019-08-14 07:45:39,370 : INFO : EPOCH 5 - PROGRESS: at 31.50% examples, 380159 words/s, in_qsize 0, out_qsize 0
2019-08-14 07:45:40,376 : INFO : EPOCH 5 - PROGRESS: at 31.78% examples, 380185 words/s, in_qsize 0, out_qsize 0
2019-08-14 07:45:41,379 : INFO : EPOCH 5 - PROGRESS: at 32.06% examples, 380175 words/s, in_qsize 0, out_qsize 0
2019-08-14 07:45:42,387 : INFO : EPOCH 5 - PROGRESS: at 32.33% examples, 380042 words/s, in_qsiz

2019-08-14 07:46:48,314 : INFO : EPOCH 5 - PROGRESS: at 50.38% examples, 380129 words/s, in_qsize 0, out_qsize 0
2019-08-14 07:46:49,361 : INFO : EPOCH 5 - PROGRESS: at 50.67% examples, 380085 words/s, in_qsize 8, out_qsize 0
2019-08-14 07:46:50,370 : INFO : EPOCH 5 - PROGRESS: at 50.97% examples, 380166 words/s, in_qsize 2, out_qsize 1
2019-08-14 07:46:51,422 : INFO : EPOCH 5 - PROGRESS: at 51.31% examples, 380327 words/s, in_qsize 5, out_qsize 0
2019-08-14 07:46:52,426 : INFO : EPOCH 5 - PROGRESS: at 51.56% examples, 380074 words/s, in_qsize 3, out_qsize 1
2019-08-14 07:46:53,429 : INFO : EPOCH 5 - PROGRESS: at 51.89% examples, 380254 words/s, in_qsize 0, out_qsize 0
2019-08-14 07:46:54,478 : INFO : EPOCH 5 - PROGRESS: at 52.18% examples, 380214 words/s, in_qsize 5, out_qsize 0
2019-08-14 07:46:55,481 : INFO : EPOCH 5 - PROGRESS: at 52.46% examples, 380339 words/s, in_qsize 4, out_qsize 0
2019-08-14 07:46:56,485 : INFO : EPOCH 5 - PROGRESS: at 52.75% examples, 380436 words/s, in_qsiz

2019-08-14 07:48:03,832 : INFO : EPOCH 5 - PROGRESS: at 69.74% examples, 371066 words/s, in_qsize 2, out_qsize 0
2019-08-14 07:48:04,836 : INFO : EPOCH 5 - PROGRESS: at 70.01% examples, 371121 words/s, in_qsize 5, out_qsize 1
2019-08-14 07:48:05,871 : INFO : EPOCH 5 - PROGRESS: at 70.28% examples, 371160 words/s, in_qsize 6, out_qsize 1
2019-08-14 07:48:06,885 : INFO : EPOCH 5 - PROGRESS: at 70.56% examples, 371270 words/s, in_qsize 7, out_qsize 0
2019-08-14 07:48:07,906 : INFO : EPOCH 5 - PROGRESS: at 70.87% examples, 371450 words/s, in_qsize 5, out_qsize 0
2019-08-14 07:48:08,911 : INFO : EPOCH 5 - PROGRESS: at 71.12% examples, 371345 words/s, in_qsize 0, out_qsize 0
2019-08-14 07:48:09,914 : INFO : EPOCH 5 - PROGRESS: at 71.39% examples, 371301 words/s, in_qsize 0, out_qsize 0
2019-08-14 07:48:10,921 : INFO : EPOCH 5 - PROGRESS: at 71.59% examples, 370990 words/s, in_qsize 0, out_qsize 0
2019-08-14 07:48:11,934 : INFO : EPOCH 5 - PROGRESS: at 71.86% examples, 371093 words/s, in_qsiz

2019-08-14 07:49:18,200 : INFO : EPOCH 5 - PROGRESS: at 88.52% examples, 368679 words/s, in_qsize 0, out_qsize 0
2019-08-14 07:49:19,205 : INFO : EPOCH 5 - PROGRESS: at 88.79% examples, 368729 words/s, in_qsize 4, out_qsize 1
2019-08-14 07:49:20,211 : INFO : EPOCH 5 - PROGRESS: at 89.07% examples, 368811 words/s, in_qsize 6, out_qsize 1
2019-08-14 07:49:21,216 : INFO : EPOCH 5 - PROGRESS: at 89.35% examples, 368918 words/s, in_qsize 4, out_qsize 0
2019-08-14 07:49:22,233 : INFO : EPOCH 5 - PROGRESS: at 89.64% examples, 369096 words/s, in_qsize 0, out_qsize 0
2019-08-14 07:49:23,636 : INFO : EPOCH 5 - PROGRESS: at 89.88% examples, 368669 words/s, in_qsize 4, out_qsize 0
2019-08-14 07:49:24,646 : INFO : EPOCH 5 - PROGRESS: at 90.14% examples, 368744 words/s, in_qsize 5, out_qsize 2
2019-08-14 07:49:25,652 : INFO : EPOCH 5 - PROGRESS: at 90.39% examples, 368815 words/s, in_qsize 6, out_qsize 0
2019-08-14 07:49:26,657 : INFO : EPOCH 5 - PROGRESS: at 90.68% examples, 369025 words/s, in_qsiz

FileNotFoundError: [Errno 2] No such file or directory: 'models/w2v/model.w2v.wv.vectors.npy'

In [19]:
model.save("models/w2v/model.w2v")

2019-08-14 07:56:41,659 : INFO : saving Word2Vec object under models/w2v/model.w2v, separately None
2019-08-14 07:56:41,665 : INFO : storing np array 'vectors' to models/w2v/model.w2v.wv.vectors.npy
2019-08-14 07:56:45,115 : INFO : not storing attribute vectors_norm
2019-08-14 07:56:45,121 : INFO : storing np array 'syn1neg' to models/w2v/model.w2v.trainables.syn1neg.npy
2019-08-14 07:56:53,674 : INFO : saved models/w2v/model.w2v


In [20]:
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences

# Высота матрицы (максимальное количество слов в твите)
SENTENCE_LENGTH = 26
# Размер словаря
NUM = 100000

def get_sequences(tokenizer, x):
    sequences = tokenizer.texts_to_sequences(x)
    return pad_sequences(sequences, maxlen=SENTENCE_LENGTH)

# Cоздаем и обучаем токенизатор
tokenizer = Tokenizer(num_words=NUM)
tokenizer.fit_on_texts(x_train)

# Отображаем каждый текст в массив идентификаторов токенов
x_train_seq = get_sequences(tokenizer, x_train)
x_test_seq = get_sequences(tokenizer, x_test)

/Users/anastasiazuhba/anaconda3/envs/Py3ML/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [23]:
from gensim.models import Word2Vec
# Загружаем обученную модель
w2v_model = Word2Vec.load('models/w2v/model.w2v')
DIM = w2v_model.vector_size 
# Инициализируем матрицу embedding слоя нулями
embedding_matrix = np.zeros((NUM, DIM))
# Добавляем NUM=100000 наиболее часто встречающихся слов из обучающей выборки в embedding слой
for word, i in tokenizer.word_index.items():
    if i >= NUM:
        break
    if word in w2v_model.wv.vocab.keys():
        embedding_matrix[i] = w2v_model.wv[word]

2019-08-14 08:00:20,252 : INFO : loading Word2Vec object from models/w2v/model.w2v
2019-08-14 08:00:25,078 : INFO : loading wv recursively from models/w2v/model.w2v.wv.* with mmap=None
2019-08-14 08:00:25,080 : INFO : loading vectors from models/w2v/model.w2v.wv.vectors.npy with mmap=None
2019-08-14 08:00:28,270 : INFO : setting ignored attribute vectors_norm to None
2019-08-14 08:00:28,301 : INFO : loading vocabulary recursively from models/w2v/model.w2v.vocabulary.* with mmap=None
2019-08-14 08:00:28,304 : INFO : loading trainables recursively from models/w2v/model.w2v.trainables.* with mmap=None
2019-08-14 08:00:28,308 : INFO : loading syn1neg from models/w2v/model.w2v.trainables.syn1neg.npy with mmap=None
2019-08-14 08:00:33,061 : INFO : loaded models/w2v/model.w2v


In [27]:
from keras import backend as K


def precision(y_true, y_pred):
    """Precision metric.

    Only computes a batch-wise average of precision.

    Computes the precision, a metric for multi-label classification of
    how many selected items are relevant.
    """
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
    precision = true_positives / (predicted_positives + K.epsilon())
    return precision


def recall(y_true, y_pred):
    """Recall metric.

    Only computes a batch-wise average of recall.

    Computes the recall, a metric for multi-label classification of
    how many relevant items are selected.
    """
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
    recall = true_positives / (possible_positives + K.epsilon())
    return recall


def f1(y_true, y_pred):
    def recall(y_true, y_pred):
        """Recall metric.

        Only computes a batch-wise average of recall.

        Computes the recall, a metric for multi-label classification of
        how many relevant items are selected.
        """
        true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
        possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
        recall = true_positives / (possible_positives + K.epsilon())
        return recall

    def precision(y_true, y_pred):
        """Precision metric.

        Only computes a batch-wise average of precision.

        Computes the precision, a metric for multi-label classification of
        how many selected items are relevant.
        """
        true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
        predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
        precision = true_positives / (predicted_positives + K.epsilon())
        return precision

    precision = precision(y_true, y_pred)
    recall = recall(y_true, y_pred)
    return 2 * ((precision * recall) / (precision + recall + K.epsilon()))

#### Свёрточная нейронная сеть

In [28]:
from keras.layers import Input
from keras.layers.embeddings import Embedding

tweet_input = Input(shape=(SENTENCE_LENGTH,), dtype='int32')
tweet_encoder = Embedding(NUM, DIM, input_length=SENTENCE_LENGTH,
                          weights=[embedding_matrix], trainable=False)(tweet_input)

In [29]:
from keras import optimizers
from keras.layers import Dense, concatenate, Activation, Dropout
from keras.models import Model
from keras.layers.convolutional import Conv1D
from keras.layers.pooling import GlobalMaxPooling1D

branches = []
# Добавляем dropout-регуляризацию
x = Dropout(0.2)(tweet_encoder)

for size, filters_count in [(2, 10), (3, 10), (4, 10), (5, 10)]:
    for i in range(filters_count):
        # Добавляем слой свертки
        branch = Conv1D(filters=1, kernel_size=size, padding='valid', activation='relu')(x)
        # Добавляем слой субдискретизации
        branch = GlobalMaxPooling1D()(branch)
        branches.append(branch)
# Конкатенируем карты признаков
x = concatenate(branches, axis=1)
# Добавляем dropout-регуляризацию
x = Dropout(0.2)(x)
x = Dense(30, activation='relu')(x)
x = Dense(1)(x)
output = Activation('sigmoid')(x)

model = Model(inputs=[tweet_input], outputs=[output])

In [30]:
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=[precision, recall, f1])
model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_3 (InputLayer)            (None, 26)           0                                            
__________________________________________________________________________________________________
embedding_2 (Embedding)         (None, 26, 200)      20000000    input_3[0][0]                    
__________________________________________________________________________________________________
dropout_3 (Dropout)             (None, 26, 200)      0           embedding_2[0][0]                
__________________________________________________________________________________________________
conv1d_41 (Conv1D)              (None, 25, 1)        401         dropout_3[0][0]                  
__________________________________________________________________________________________________
conv1d_42 

In [31]:
from keras.callbacks import ModelCheckpoint

checkpoint = ModelCheckpoint("models/cnn/cnn-frozen-embeddings-{epoch:02d}-{val_f1:.2f}.hdf5", monitor='val_f1', save_best_only=True, mode='max', period=1)
history = model.fit(x_train_seq, y_train, batch_size=32, epochs=10, validation_split=0.25, callbacks = [checkpoint])

Train on 134307 samples, validate on 44769 samples
Epoch 1/10
134307/134307 [==============================] - 385s 3ms/step - loss: 0.5719 - precision: 0.6992 - recall: 0.6937 - f1: 0.6874 - val_loss: 0.5082 - val_precision: 0.7167 - val_recall: 0.8122 - val_f1: 0.7558
Epoch 2/10
134307/134307 [==============================] - 366s 3ms/step - loss: 0.5170 - precision: 0.7411 - recall: 0.7315 - f1: 0.7288 - val_loss: 0.4887 - val_precision: 0.7639 - val_recall: 0.7484 - val_f1: 0.7496
Epoch 3/10
134307/134307 [==============================] - 354s 3ms/step - loss: 0.5038 - precision: 0.7496 - recall: 0.7403 - f1: 0.7376 - val_loss: 0.4860 - val_precision: 0.7972 - val_recall: 0.6951 - val_f1: 0.7358
Epoch 4/10
134307/134307 [==============================] - 348s 3ms/step - loss: 0.4965 - precision: 0.7548 - recall: 0.7438 - f1: 0.7422 - val_loss: 0.4806 - val_precision: 0.7542 - val_recall: 0.7838 - val_f1: 0.7631
Epoch 5/10
134307/134307 [==============================] - 356s 3ms/

Затем выбираем модель с наивысшими показателями F-меры на валидационном наборе данных, т.е. модель, полученную на восьмой эпохе обучения. У модели разморозил embedding-слой, после чего запустил еще пять эпох обучения. 

In [33]:
from keras import optimizers
# Загружаем веса модели
model.load_weights('models/cnn/cnn-frozen-embeddings-06-0.77.hdf5')
# Делаем embedding слой способным к обучению
model.layers[1].trainable = True
# Уменьшаем learning rate
adam = optimizers.Adam(lr=0.0001)
model.compile(loss='binary_crossentropy', optimizer=adam, metrics=[precision, recall, f1])
model.summary()
 

checkpoint = ModelCheckpoint("models/cnn/cnn-trainable-{epoch:02d}-{val_f1:.2f}.hdf5", monitor='val_f1', save_best_only=True, mode='max', period=1)

history_trainable = model.fit(x_train_seq, y_train, batch_size=32, epochs=5, validation_split=0.25, callbacks = [checkpoint])

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_3 (InputLayer)            (None, 26)           0                                            
__________________________________________________________________________________________________
embedding_2 (Embedding)         (None, 26, 200)      20000000    input_3[0][0]                    
__________________________________________________________________________________________________
dropout_3 (Dropout)             (None, 26, 200)      0           embedding_2[0][0]                
__________________________________________________________________________________________________
conv1d_41 (Conv1D)              (None, 25, 1)        401         dropout_3[0][0]                  
__________________________________________________________________________________________________
conv1d_42 

Train on 134307 samples, validate on 44769 samples
Epoch 1/5
  6112/134307 [>.............................] - ETA: 36:04 - loss: 0.4774 - precision: 0.7631 - recall: 0.7648 - f1: 0.7585

KeyboardInterrupt: 